In [4]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm 
import imageio
from albumentations import HorizontalFlip, VerticalFlip, ElasticTransform, OpticalDistortion

In [5]:
def createdir(path):
  if (not os.path.exists(path)):
    os.mkdir(path)

In [6]:
def load_data(path):
  train_x= sorted(glob(os.path.join(psth, "Training", ""  "*.tif")))
  train_y= sorted(glob(os.path.join(path, "Training", "", "*.gif")))

  test_x= sorted(glob(os.path.join(psth, "testing", ""  "*.tif")))
  test_y= sorted(glob(os.path.join(path, "testing", "", "*.gif")))

  return (train_x, train_y), (test_x, test_y)
  

In [ ]:
def augment_data(images, masks, save_path, augment=True):

  # specify the dimensions of the image #
  h= 224
  w= 224

  for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):

    # extracting names #
    name= x.split("/")[-1].split(".")[0]

    # The above code is specific to the dataset used, and must be used accordingly #

    # Reading images and masks #
    
    x= cv2.imread(x, cv2.IMREAD_COLOR)
    y= imageio.mimread(y)[0] #used imageio since the masks are ing gif format #

    if augment== True :
      aug = HorizontalFlip(p=1.0)
      augmented= aug(image=x ,mask=y)
      x1 = augmented["image"]
      y1 = augmented["mask"]

      aug = VerticalFlip(p=1.0)
      augmented= aug(image=x ,mask=y)
      x2 = augmented["image"]
      y2 = augmented["mask"]

      aug = ElasticTransform(p=1, alpha=120*0.05, alpha_affine=120*0.03)
      augmented= aug(image=x ,mask=y)
      x3 = augmented["image"]
      y3 = augmented["mask"]

      aug = OpticalDistortion(p=1, distort_limit=2, shift_limit=0.5)
      augmented= aug(image=x ,mask=y)
      x4 = augmented["image"]
      y4 = augmented["mask"]

      X= [x, x1, x2, x3, x4]
      Y= [y, y1, y2, y3, y4]

      else:
        X=[x]
        Y=[y]

      index= 0
      for i, m in zip(X, Y):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))

            if len(X) == 1:
                tmp_image_name = f"{name}.jpg"
                tmp_mask_name = f"{name}.jpg"
            else:
                tmp_image_name = f"{name}_{index}.jpg"
                tmp_mask_name = f"{name}_{index}.jpg"

            image_path = os.path.join(save_path, "image", tmp_image_name)
            mask_path = os.path.join(save_path, "mask", tmp_mask_name)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            index += 1       
